In [1]:
import subprocess

def run_sim(simTime, appPeriod, nDevices, dataUpType, realisticModel, endDeviceType, rx1FRx2):
    ns3_script = "lora-device-classes-example"

    cmd_params = " --useRx1ParamsForRx2ClassC="+f"{rx1FRx2}"+" --endDeviceType="+f"{endDeviceType}"+" --appPeriod="+f"{appPeriod}"+" --simulationTime="+f"{simTime}"+" --nDevices="+f"{nDevices}"+" --dataUpType="+f"{dataUpType}"+" --realisticModel="+f"{realisticModel}"

    command = "/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "+ f'"{ns3_script + cmd_params}"'
    print(command)
    result=""
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, text=True).stdout.splitlines()[-1]
    except subprocess.CalledProcessError as e:
        print(f"Error running script: {e}")
    
    return result

In [2]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message, to):
    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login('phani.jsp@gmail.com', 'rkyq vvjd xtwh kfwx')
        
        msg = MIMEText(message)
        msg['Subject'] = subject
        msg['From'] = 'vlad@Castlevania.com'
        msg['To'] = to
        
        server.sendmail('vlad@Castlevania', to, msg.as_string())
        server.quit()
        print('Successfully sent email to %s:' % (to))
    except Exception as e:
        print('Failed to send email to %s:' % (to))
        print(e)

In [3]:
import numpy as np

def devices_log_list():
    initial_value = 1
    #Lambda value 5, corresponds to almost 500 devices at current app period of 120
    final_value = 4500
    num_points = 40
    log_values = np.logspace(np.log10(initial_value), np.log10(final_value), num=num_points)
    # log_values = np.linspace(initial_value, final_value, num=num_points)
    return log_values

In [4]:
# for i  in devices_log_list():
#     print(max(1,round(i)))


In [5]:
import concurrent.futures
import json
import math
import os


def run_scenario(traffic_type, realistic_model, device_class, scenario_name, rx1FRx2):
    
    simTime = 800
    appPeriod = 600
    data_all = []
    futures = []

    with concurrent.futures.ThreadPoolExecutor() as executor:

        deviceLogIter = devices_log_list()
        for i in deviceLogIter:
            deviceCount = max(1,round(i))
            print(scenario_name+", Simualtion with device count: "+str(deviceCount))
            future = executor.submit(run_sim, simTime, appPeriod, deviceCount, traffic_type, realistic_model, device_class, rx1FRx2)
            futures.append(future)



    for future in futures:
        data = json.loads(future.result())
        if float(data["ULPDR"]["Total"])>0:
            data_all.append(data)
            print(str(float(data["ULPDR"]["Success"])/float(data["ULPDR"]["Total"]))+" L: "+str(float(data["ULPDR"]["Total"])/float(simTime)))
        
    test_folder_name = 'test_scenario_rx1_for_rx2_class_c'
    test_sub_folder = str(rx1FRx2)

    target_folder = os.path.join(test_folder_name, test_sub_folder)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    filePath = os.path.join(target_folder, scenario_name)

    with open(filePath, 'w') as f:
        json.dump(data_all, f)

In [6]:
import threading

threads = []


In [7]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", False)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", False)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", False)))

In [8]:

threads.append(threading.Thread(target=run_scenario,args=(0, False, 'C', "ULPDR_UNCONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(1, False, 'C', "ULPDR_CONFIRMED_C", True)))
threads.append(threading.Thread(target=run_scenario,args=(2, False, 'C', "ULPDR_MIXED_C", True)))

In [9]:
import time
start_time = time.time()
for thread in threads:
    thread.start()

for thread in threads:
    thread.join()
end_time = time.time()

elapsed_time = end_time - start_time
print(elapsed_time)
send_email('[IMP] Sim Plot Run Status Rx1_f_Rx2', "Simulation complete!.\nTotal run time: {:.6f} seconds.\nPlease check the results on your vm.\n\nThank you,\nVlad".format(elapsed_time), 'aditya.jagatha@trojans.dsu.edu')

ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
ULPDR_CONFIRMED_C, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=0 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 1
ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
ULPDR_UNCONFIRMED_C, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=1 --realisticModel=False"
ULPDR_CONFIRMED_C, Simualtion with device count: 1
ULPDR_CONFIRMED_C, Simualtion with device count: 1
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=1 --dataUpType=2

+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Running simulation...
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
Computing performance metrics...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=6 --dataUpType=0 --realisticModel=False"ULPDR_UNCONFIRMED_C, Simualtion with device count: 7
ULPDR_CONFIRMED_C, Simualtion with device count: 7
ULPDR_CONFIRMED_C, Simualtion with device count: 9

/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=6 --dataUpType=1 --realisticModel=False"
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=7 --dataUpType=1 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 7
ULPDR_UNCONFIRMED_C, Simualtion with device count: 9
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example

+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
Computing performance metrics...
+0.000000000s +0.000000000Completed configurations
 Running simulation...
+0.000000000s Completed configuration
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration
+0.000000000s Running simulation...
Computing performance metrics...
+0.000000000s Completed configuration

ULPDR_UNCONFIRMED_C, Simualtion with device count: 20ULPDR_UNCONFIRMED_C, Simualtion with device count: 17
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=13 --dataUpType=0 --realisticModel=False"

ULPDR_CONFIRMED_C, Simualtion with device count: 20
ULPDR_CONFIRMED_C, Simualtion with device count: 17
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=False --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=17 --dataUpType=1 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 17
/home/adi/Workspace/ns-allinone-3.37/ns-3.37/ns3 run "lora-device-classes-example --useRx1ParamsForRx2ClassC=True --endDeviceType=C --appPeriod=600 --simulationTime=800 --nDevices=13 --dataUpType=2 --realisticModel=False"
ULPDR_MIXED_C, Simualtion with device count: 20
ULPDR_UNCONFIRMED_C, Simu

...

1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
0.9 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
0.9558823529411765 L: 0.085
0.9518072289156626 L: 0.10375
0.9583333333333334 L: 0.12
0.9669421487603306 L: 0.15125
0.9403973509933775 L: 0.18875
0.9132653061224489 L: 0.245
0.9083333333333333 L: 0.3
0.8782894736842105 L: 0.38
0.8854748603351955 L: 0.4475
0.8681818181818182 L: 0.55
0.8628158844765343 L: 0.6925
0.869309838472834 L: 0.85125
0.8341346153846154 L: 1.04
0.8213592233009709 L: 1.2875
0.8026835043409629 L: 1.58375
0.7909033480732786 L: 1.97875
0.7493658041603247 L: 2.46375
0.7103533278553821 L: 3.0425
0.6614927344782034 L: 3.785
0.6146445750935329 L: 4.6775
0.5330722367275892 L: 5.745
0.4742448053081893 L: 7.15875


Computing performance metrics...
Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
0.9 L: 0.025
0.9523809523809523 L: 0.02625
1.0 L: 0.0325
0.96875 L: 0.04
1.0 L: 0.0575
0.9607843137254902 L: 0.06375
0.8823529411764706 L: 0.085
0.9759036144578314 L: 0.10375
0.9574468085106383 L: 0.1175
0.9745762711864406 L: 0.1475
1.0 L: 0.1825
0.989010989010989 L: 0.2275
0.9866071428571429 L: 0.28
0.9928825622775801 L: 0.35125
0.9939759036144579 L: 0.415
0.9927360774818402 L: 0.51625
0.9941060903732809 L: 0.63625
0.9906687402799378 L: 0.80375
0.9847133757961783 L: 0.98125
0.9794661190965093 L: 1.2175
0.9670886075949368 L: 1.48125
0.9461794019933555 L: 1.88125
0.9120643431635389 L: 2.33125
0.8797221016066001 L: 2.87875
0.826996860830136 L: 3.58375
0.7616246498599439 L: 4.4625
0.6829714285714286 L: 5.46875
0.5963269054178145 L: 6.80625


Computing performance metrics...


1.0 L: 0.0025
1.0 L: 0.0025
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00375
1.0 L: 0.00625
1.0 L: 0.00875
1.0 L: 0.00875
1.0 L: 0.0075
1.0 L: 0.01
1.0 L: 0.01625
1.0 L: 0.02
1.0 L: 0.025
1.0 L: 0.02625
1.0 L: 0.0325
1.0 L: 0.04
1.0 L: 0.0575
1.0 L: 0.06375
1.0 L: 0.08375
1.0 L: 0.10375
1.0 L: 0.11875
1.0 L: 0.14625
1.0 L: 0.185
1.0 L: 0.23125
0.9914163090128756 L: 0.29125
0.9965034965034965 L: 0.3575
1.0 L: 0.42625
0.990521327014218 L: 0.5275
0.9766990291262136 L: 0.64375
0.970679012345679 L: 0.81
0.9572327044025157 L: 0.99375
0.9663265306122449 L: 1.225
0.9394449116904963 L: 1.48625
0.9075297225891678 L: 1.8925
0.8634903640256959 L: 2.335
0.8327571305099395 L: 2.8925
0.7376650451702571 L: 3.5975
0.6528787031861375 L: 4.4725
0.5453299840146152 L: 5.47375
0.46945514584479914 L: 6.81375
4140.8566172122955
Successfully sent email to aditya.jagatha@trojans.dsu.edu:
